<a href="https://colab.research.google.com/github/Uzmamushtaque/CSCI4962-Projects-ML-AI/blob/main/Lecture_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 14

# Today's Topics

1. K-Means Clustering
2. Self-Organizing Maps (SOMs)

There are two concepts that form the basic building block of SOMs: Clustering data, Competitive Learning.

# K-Means Clustering

Clustering is the process of dividing the entire data into groups (also known as clusters) based on the patterns in the data.

An important property of clusters is that the points within a cluster should be similar to each other. The primary aim here is to minimize the distance between the points within a cluster.

K-means is a centroid-based algorithm, or a distance-based algorithm, where we calculate the distances to assign a point to a cluster. In K-Means, each cluster is associated with a centroid.

The main objective of the K-Means algorithm is to minimize the sum of distances between the points and their respective cluster centroid.

The Algorithm:

Step 1: Choose the number of clusters k

Step 2: Select k random points from the data as centroids

Step 3: Assign all the points to the closest cluster centroid

Step 4: Recompute the centroids of newly formed clusters

Step 5: Repeat steps 3 and 4

### Stopping Criteria for K-Means Clustering

There are essentially three stopping criteria that can be adopted to stop the K-means algorithm:

1. Centroids of newly formed clusters do not change

2. Points remain in the same cluster

3. Maximum number of iterations are reached

# Competitive learning

Until now, for all Neural networks we have been updating weights to correct for errors. Under competitive learning paradigm, the goal is not to map inputs to outputs (to correct errors). Rather the neurons compete for the right to respond to a subset of similar input data and push their weights closer to one of the input points. Learning process is also different from Backpropagation in Neural Networks.

General Idea:

Activation of a neuron increases with greater similarity between the weight vector of the neuron and the input. It is assumed the the weight vector has the same dimensionality as the input. A common approach is to use the Euclidian distance between the input and the weight vector in order to compute the activation. Smaller distances lead to larger activation. The output unit that has the highest activation to a given input is declared the winner and moved closer to the input.

# SOMs

A self-organizing map (SOM) or self-organizing feature map (SOFM) is an unsupervised machine learning technique used to produce a low-dimensional (typically two-dimensional) representation of a higher dimensional data set while preserving the topological structure of the data.

[Source](https://en.wikipedia.org/wiki/Self-organizing_map)